In [6]:
from scipy import io
import numpy as np
from functional import seq
import os
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize

%matplotlib inline

sift_loc = "/data/yangyang/FlyExpress_SIFT_feature/"
img_index = "/data/yangyang/SIFT/30_labels.pickle"

In [7]:
def read_all_file(folder):
    data = []
    for file in os.listdir(folder):
        name = os.fsdecode(file)
        if name.endswith('.mat'):
            mat = io.loadmat(folder+file)
            data.append(mat['FlyExpress_SIFT_feature'][0, :])
    return np.hstack(data)

def unique(idx, image):
    idx, slice = np.unique(idx, return_index=True)
    return idx, image[slice, :]

def separate_set(sift_loc, index_loc):
    mat_data = read_all_file(sift_loc)
    with open(index_loc, 'rb') as f:
        idx = pickle.load(f)
        train_idx, test_idx = set(idx['train']), set(idx['test'])
        
    print(len(train_idx))
    print(len(test_idx))
    
    train_stream = seq(mat_data).smap(lambda idx, data: (idx[0], data.flatten())) \
        .filter(lambda dp: dp[0] in train_idx) \
        .list()
    train_idx, train_img = zip(*train_stream)
    train_idx = np.array(train_idx)
    train_img = np.vstack(train_img)
    train_idx, train_img = unique(train_idx, train_img)
    
    test_stream = seq(mat_data).smap(lambda idx, data: (idx[0], data.flatten())) \
        .filter(lambda dp: dp[0] in test_idx) \
        .list()
    test_idx, test_img = zip(*test_stream)
    test_idx = np.array(test_idx)
    test_img = np.vstack(test_img)
    test_idx, test_img = unique(test_idx, test_img)
    
    return train_idx, train_img, test_idx, test_img

def to_dict(train_idx, train_img, test_idx, test_img):
    train = seq(train_idx).zip(seq(train_img)).dict()
    test = seq(test_idx).zip(seq(test_img)).dict()
    return {**train, **test}

In [9]:
train_idx, train_img, test_idx, test_img = separate_set(sift_loc, img_index)

pca = PCA(n_components=512, whiten=True)
#normalizer = Normalizer(axis=0)
train_norm = normalize(train_img, norm='l1', axis=0)
test_norm = normalize(test_img, norm='l1', axis=0)
train_feature = pca.fit_transform(train_norm)
test_feature = pca.transform(test_norm)
data = to_dict(train_idx, train_feature, test_idx, test_feature)
with open('30_labels_feature.pickle', 'wb') as f:
    pickle.dump(data, f)

17469
17181


In [10]:
pca.explained_variance_ratio_.sum()

0.99928809178113398

In [ ]:
data.keys()